<a href="https://colab.research.google.com/github/joshibasu62/lunar_landing/blob/main/notebooks/unit1/unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install swig cmake

In [ ]:
!pip install pygame --pre --no-cache-dir


In [ ]:
!pip install stable-baselines3==2.0.0a5 swig huggingface_sb3 gymnasium==0.28.1 box2d-py==2.3.5


In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [ ]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Let's see what the Environment looks like:


In [ ]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

In [ ]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

In [ ]:
# TODO: Define a PPO MlpPolicy architecture
# We use MultiLayerPerceptron (MLPPolicy) because the input is a vector,
# if we had frames as input we would use CnnPolicy
# Instantiate the agent
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.99,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)
# Train the agent
# model.learn(total_timesteps=int(2e5))

In [ ]:
# TODO: Train it for 1,000,000 timesteps
model.learn(total_timesteps = 1000000)
# TODO: Specify file name for model and save the model to file
model_name = "ppo-LunarLander-v2"
model.save(model_name)


In [ ]:
# TODO: Evaluate the agent
# Create a new environment for evaluation
eval_env = Monitor(gym.make('LunarLander-v2', render_mode = 'rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")




In [ ]:

# Record a video of the trained agent


from gymnasium.wrappers import RecordVideo
from IPython.display import HTML
from base64 import b64encode
import os

# Wrap environment to record video
video_env = RecordVideo(
    gym.make("LunarLander-v2", render_mode="rgb_array"),
    video_folder="videos/",
    episode_trigger=lambda episode_id: True  # record all episodes
)

obs, info = video_env.reset()
done = False

while not done:
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = video_env.step(action)
    done = terminated or truncated

video_env.close()

# Find the recorded video file
video_path = sorted([f for f in os.listdir("videos/") if f.endswith(".mp4")])[-1]

# Display the video in Colab
mp4 = open(f"videos/{video_path}", 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=600 controls><source src="{data_url}" type="video/mp4"></video>')
